In [196]:
import os
import numpy as np
from dfply import *
import pandas as pd
from datetime import datetime

In [426]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format

In [427]:
# 데이터 불러오기
금감원_금리시나리오 = pd.read_excel('./data/현행추정부채/금감원_금리시나리오.xlsx') # 상승/하락/평행/경사 구분코드 넣기
일반_상품군정보 = pd.read_excel('./data/현행추정부채/일반_상품군정보.xlsx', dtype={'PDGR_CD': str})
일반_약관정보 = pd.read_excel('./data/현행추정부채/일반_약관정보.xlsx', dtype={'ARC_INPL_CD': str, 'PDC_CD': str, 'PDGR_CD': str})
일반_원수_개별추산액 = pd.read_excel('./data/현행추정부채/일반_원수_개별추산액.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_원수_미경과보험료 = pd.read_excel('./data/현행추정부채/일반_원수_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_납입예정보험료 = pd.read_excel('./data/현행추정부채/일반_납입예정보험료.xlsx', dtype={'BSE_YM': str, 'ARC_INPL_CD': str, 'PYM_EPCT_YM': str})
일반_출재_개별추산액 = pd.read_excel('./data/현행추정부채/일반_출재_개별추산액.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_출재_미경과보험료 = pd.read_excel('./data/현행추정부채/일반_출재_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_직전1년경과보험료 = pd.read_excel('./data/현행추정부채/일반_원수_직전1년경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_출재_직전1년경과보험료 = pd.read_excel('./data/현행추정부채/일반_출재_직전1년경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_연동수수료특약코드 = pd.read_excel('./data/현행추정부채/일반_연동수수료특약코드.xlsx')
일반_진전추이 = pd.read_excel('./data/현행추정부채/일반_진전추이.xlsx', dtype={'BSE_YM': str, 'PDGR_CD': str, 'AY_YM': str})
국가_중분류코드 = pd.read_excel('./data/현행추정부채/국가_중분류코드.xlsx', dtype={'CNTR_CATG_CD': str})
국가_대분류코드 = pd.read_excel('./data/현행추정부채/국가_대분류코드.xlsx', dtype={'CNTR_CATG_CD': str, 'KICS_CNTR_CATG_CD': str})
일반_원수_손해조사비율 = pd.read_excel('./data/현행추정부채/일반_원수_손해조사비율.xlsx', dtype={'CLG_YM': str, 'RRNR_DAT_DVCD': str, 'PDGR_CD': str, 'PDC_CD': str})
일반_출재_손해조사비율 = pd.read_excel('./data/현행추정부채/일반_출재_손해조사비율.xlsx', dtype={'CLG_YM': str, 'RRNR_DAT_DVCD': str, 'PDGR_CD': str, 'PDC_CD': str})
회수율 = pd.read_excel('./data/현행추정부채/회수율.xlsx', dtype={'ASUM_INPD_DVCD': str})
선도부도율 = pd.read_excel('./data/현행추정부채/선도부도율.xlsx', dtype={'BASE_YYMM': str, 'CRD_GRD_CD': str})
신용등급분류 = pd.read_excel('./data/현행추정부채/신용등급분류.xlsx', dtype={'CRD_GRD_CD': str})
재보험자_국내신용등급 = pd.read_excel('./data/현행추정부채/재보험자_국내신용등급.xlsx', dtype={'T02_RN_RINSC_CD': str})

#### 1. 일반/원수/계약 - 미경과보험료

In [459]:
# 일반 원수 계약 PDGR_NM 가공
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 원수 계약 DMFR_DVCD 가공
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [460]:
# 일반 원수 계약 미경과보험료 집계
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(OGL_URND_PRM=np.sum(X.OGL_URND_PRM)) \
    >> mutate(OGL_URND_PRM=X.OGL_URND_PRM/1e3)
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_집계.reset_index(drop=True)
일반_원수_미경과보험료_집계

,DMFR_DVCD,PDGR_NM,OGL_URND_PRM
0,국내,일반/근재,"6,997,299.7560"
1,해외,일반/근재,"4,386,874.5990"
2,국내,일반/기술,"29,494,169.0440"
3,해외,일반/기술,"1,771,534.3690"
4,국내,일반/기타,"38,447,006.5220"
5,해외,일반/기타,"623,938.8020"
6,국내,일반/농작물재해보상,"7,347,601.1820"
7,국내,일반/배상,"66,489,197.5320"
8,해외,일반/배상,"14,435,185.2600"
9,국내,일반/보증,"11,066.4040"


#### 2. 일반/출재/계약 - 미경과보험료

In [464]:
# 일반 출재 계약 PDGR_NM 가공
일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> left_join(재보험자_국내신용등급, on='T02_RN_RINSC_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 계약 DMFR_DVCD 가공
일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [466]:
# 일반 출재 계약 미경과보험료 집계
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_가공 >> group_by(X.GRADE, X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(RN_URND_PRM=np.sum(X.RN_URND_PRM)) \
    >> mutate(RN_URND_PRM=X.RN_URND_PRM/1e3)
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_집계.reset_index(drop=True)
일반_출재_미경과보험료_집계

,DMFR_DVCD,PDGR_NM,GRADE,RN_URND_PRM
0,국내,일반/기술,A,"1,733.3400"
1,국내,일반/배상,A,"6,147.5620"
2,국내,일반/종합,A,"144,047.0890"
3,국내,일반/해상,A,"84,983.8870"
4,국내,일반/근재,AA,"1,417,380.3700"
5,국내,일반/기술,AA,"14,756,178.2050"
6,국내,일반/기타,AA,"12,278,414.6620"
7,국내,일반/농작물재해보상,AA,"4,408,560.7100"
8,국내,일반/배상,AA,"22,805,363.0210"
9,국내,일반/보증,AA,"10,513.0840"


#### 3. 할인율

In [432]:
금감원_금리시나리오_가공 = 금감원_금리시나리오 \
    >> mutate(MATURITY=X.BUCKET_END_DATE.dt.year*12 + X.BUCKET_END_DATE.dt.month - X.AS_OF_DATE.dt.year*12 - X.AS_OF_DATE.dt.month) \
    >> mutate(DISCOUNT=1/(1+X.VALUE)**(X.MATURITY/12)) \
    >> select(X.MATURITY, X.DISCOUNT)

#### 4. 일반/원수/계약 - 미래보험료

In [433]:
# 일반 계약 납입예정보험료 MATURITY 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료 >> left_join(일반_약관정보[['ARC_INPL_CD', 'PDGR_CD']], on='ARC_INPL_CD') \
    >> mutate(PYM_EPCT_YM=pd.to_datetime(일반_납입예정보험료['PYM_EPCT_YM'], format='%Y%m')) \
    >> mutate(BSE_YM=pd.to_datetime(일반_납입예정보험료['BSE_YM'], format='%Y%m')) \
    >> mutate(MATURITY=X.PYM_EPCT_YM.dt.year*12 + X.PYM_EPCT_YM.dt.month - X.BSE_YM.dt.year*12 - X.BSE_YM.dt.month) \
    >> drop(X.BSE_YM, X.PYM_EPCT_YM) \
    >> mutate(MATURITY=if_else(X.MATURITY < 0, 0, X.MATURITY))

# 일반 계약 납입예정보험료 DMFR_DVCD 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

# 일반 계약 납입예정보험료 PDGR_NM 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDC_CD=X.ARC_INPL_CD.str.slice(0,5)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 할인율 반영
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> left_join(금감원_금리시나리오_가공, on='MATURITY') \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM*X.DISCOUNT) \
    >> mutate(WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM*X.DISCOUNT)

In [434]:
# 일반 계약 납입예정보험료 집계
일반_납입예정보험료_집계 = 일반_납입예정보험료_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(WNCR_CNV_EPCT_PRM_DISC=np.sum(X.WNCR_CNV_EPCT_PRM_DISC), WNCR_CNV_RN_PRM_DISC=np.sum(X.WNCR_CNV_RN_PRM_DISC)) \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM_DISC/1e3, WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM_DISC/1e3)
일반_납입예정보험료_집계 = 일반_납입예정보험료_집계.reset_index(drop=True)
일반_납입예정보험료_집계

,DMFR_DVCD,PDGR_NM,WNCR_CNV_EPCT_PRM_DISC,WNCR_CNV_RN_PRM_DISC
0,국내,일반/근재,"891,585","167,119"
1,국내,일반/기술,"1,290,419","642,444"
2,국내,일반/기타,"13,706","9,594"
3,국내,일반/배상,"2,877,078","843,306"
4,국내,일반/보증,"84,745","80,508"
5,국내,일반/상해,"14,336,685","2,883,026"
6,국내,일반/종합,"14,489,146","1,560,491"
7,국내,일반/해상,"25,565,519","10,440,624"


#### 5. 일반/원수/보상 - 개별추산액

In [435]:
일반_회수율 = 회수율 >> mask(X.ASUM_INPD_DVCD == '1') \
    >> mask(X.KEY.str.slice(-2) == 'ZZ') \
    >> drop(X.LAST_MODIFIED_BY, X.LAST_UPDATE_DATE) \
    >> mutate(RRNR_DVCD=X.KEY.str.slice(3,5)) \
    >> mutate(PDGR_CD=X.KEY.str.slice(-5, -3)) \

일반_원수_회수율 = 일반_회수율 >> mask(X.RRNR_DVCD.isin(['01','02']))
일반_출재_회수율 = 일반_회수율 >> mask(X.RRNR_DVCD=='03')

In [436]:
# 일반 원수 보상 PDGR_NM 가공
일반_원수_개별추산액_가공 = 일반_원수_개별추산액 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM)) \
    >> mutate(RRNR_DVCD=if_else(X.RRNR_DAT_DVCD=='01', '01', '02')) \
    >> left_join(일반_원수_회수율[['RRNR_DVCD', 'PDGR_CD', 'RCVR_RT']], on=['RRNR_DVCD', 'PDGR_CD']) \
    >> mutate(RCVR_RT=if_else(X.RCVR_RT.isna(), 0, X.RCVR_RT)) \
    >> mutate(OST_AMT_RCVR=X.OST_AMT*(1-X.RCVR_RT))

# 일반 원수 보상 DMFR_DVCD 가공
일반_원수_개별추산액_가공 = 일반_원수_개별추산액_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [437]:
# 일반 원수 보상 개별추산액 집계
일반_원수_개별추산액_집계 = 일반_원수_개별추산액_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT), OST_AMT_RCVR=np.sum(X.OST_AMT_RCVR)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3, OST_AMT_RCVR=X.OST_AMT_RCVR/1e3)
일반_원수_개별추산액_집계 = 일반_원수_개별추산액_집계.reset_index(drop=True)
일반_원수_개별추산액_집계

,DMFR_DVCD,PDGR_NM,OST_AMT,OST_AMT_RCVR
0,국내,일반/근재,"8,516,242","8,268,845"
1,해외,일반/근재,"4,176,001","4,054,688"
2,국내,일반/기술,"28,608,543","28,553,901"
3,해외,일반/기술,"925,406","923,638"
4,국내,일반/기타,"16,228,978","16,123,165"
5,국내,일반/농작물재해보상,"3,831,950","3,806,966"
6,해외,일반/농작물재해보상,0,0
7,국내,일반/배상,"46,768,724","45,852,057"
8,해외,일반/배상,"48,319,847","47,372,778"
9,국내,일반/보증,"251,736","250,094"


#### 6. 일반/출재/보상 - 개별추산액

In [438]:
# 일반 출재 보상 PDGR_NM 가공
일반_출재_개별추산액_가공 = 일반_출재_개별추산액 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM)) \
    >> left_join(일반_출재_회수율[['PDGR_CD', 'RCVR_RT']], on=['PDGR_CD']) \
    >> mutate(RCVR_RT=if_else(X.RCVR_RT.isna(), 0, X.RCVR_RT)) \
    >> mutate(RN_OST_AMT_RCVR=X.RN_OST_AMT*(1-X.RCVR_RT))

# 일반 출재 보상 DMFR_DVCD 가공
일반_출재_개별추산액_가공 = 일반_출재_개별추산액_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [439]:
# 일반 출재 보상 개별추산액 집계
일반_출재_개별추산액_집계 = 일반_출재_개별추산액_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT), RN_OST_AMT_RCVR=np.sum(X.RN_OST_AMT_RCVR)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3, RN_OST_AMT_RCVR=X.RN_OST_AMT_RCVR/1e3)
일반_출재_개별추산액_집계 = 일반_출재_개별추산액_집계.reset_index(drop=True)
일반_출재_개별추산액_집계

,DMFR_DVCD,PDGR_NM,RN_OST_AMT,RN_OST_AMT_RCVR
0,국내,일반/근재,"1,771,726","1,720,257"
1,국내,일반/기술,"18,556,329","18,520,887"
2,국내,일반/기타,"6,564,878","6,522,075"
3,국내,일반/농작물재해보상,"2,394,430","2,378,818"
4,해외,일반/농작물재해보상,0,0
5,국내,일반/배상,"26,342,298","25,825,989"
6,국내,일반/보증,"239,149","237,590"
7,국내,일반/상해,"1,220,335","1,220,274"
8,국내,일반/종합,"57,247,450","55,732,648"
9,해외,일반/종합,"43,298,811","42,482,725"


#### 6-2. 일반/원수/보상 - 손해조사비율

In [440]:
# 일반 원수 보상 PDGR_NM 가공
일반_원수_손해조사비율_가공 = 일반_원수_손해조사비율 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 원수 보상 DMFR_DVCD 가공
일반_원수_손해조사비율_가공 = 일반_원수_손해조사비율_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [441]:
# 일반 원수 손해조사비율 산출
일반_원수_손해조사비율_집계 = 일반_원수_손해조사비율_가공 >> group_by('PDGR_NM', 'DMFR_DVCD') \
    >> summarise(PYN_BNF=np.sum(X.PYN_BNF), LSAT=np.sum(X.LSAT)) \
    >> mutate(PYN_BNF=X.PYN_BNF/1e3, LSAT=X.LSAT/1e3)
일반_원수_손해조사비율_집계

,DMFR_DVCD,PDGR_NM,PYN_BNF,LSAT
0,국내,일반/근재,"52,402,868","2,835,433"
1,해외,일반/근재,"10,325,967","3,721,080"
2,국내,일반/기술,"33,797,938","1,369,330"
3,해외,일반/기술,"1,254,959","95,220"
4,국내,일반/기타,"224,035,161","3,684,270"
5,해외,일반/기타,"2,338","1,651"
6,국내,일반/농작물재해보상,"49,907,132","1,372,145"
7,해외,일반/농작물재해보상,"22,571,420","791,777"
8,국내,일반/배상,"144,450,125","20,023,640"
9,해외,일반/배상,"76,698,003","12,224,599"


#### 6-3. 일반/출재/보상 - 손해조사비율

In [467]:
# 일반 출재 보상 PDGR_NM 가공
일반_출재_손해조사비율_가공 = 일반_출재_손해조사비율 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 보상 DMFR_DVCD 가공
일반_출재_손해조사비율_가공 = 일반_출재_손해조사비율_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [468]:
# 일반 출재 손해조사비율 산출
일반_출재_손해조사비율_집계 = 일반_출재_손해조사비율_가공 >> group_by('PDGR_NM', 'DMFR_DVCD') \
    >> summarise(RN_PYN_BNF=np.sum(X.RN_PYN_BNF), RN_LSAT=np.sum(X.RN_LSAT)) \
    >> mutate(RN_PYN_BNF=X.RN_PYN_BNF/1e3, RN_LSAT=X.RN_LSAT/1e3)
일반_출재_손해조사비율_집계

,DMFR_DVCD,PDGR_NM,RN_PYN_BNF,RN_LSAT
0,국내,일반/근재,"9,677,738.7450","578,447.8430"
1,국내,일반/기술,"19,485,908.0350","800,697.6660"
2,국내,일반/기타,"101,431,452.7960","1,714,664.2700"
3,국내,일반/농작물재해보상,"26,246,282.7650","823,286.7610"
4,해외,일반/농작물재해보상,"14,459,308.7360","712,598.8680"
5,국내,일반/배상,"73,278,660.7080","9,213,850.5710"
6,국내,일반/보증,0.0000,0.0000
7,국내,일반/상해,"225,929,368.3550","7,745,500.7440"
8,국내,일반/종합,"112,072,571.7800","5,351,748.7480"
9,해외,일반/종합,"229,699,416.5710","21,844,550.4970"


#### 7. 할인요소

In [469]:
일반_진전추이_가공 = 일반_진전추이.copy()

할인요소 = []
for PDGR_CD in 일반_진전추이_가공['PDGR_CD'].unique():
    일반_진전추이_보종 = 일반_진전추이_가공.query('PDGR_CD == @PDGR_CD').sort_values(by='AY_YM')
    할인요소_보종 = []
    m = len(일반_진전추이_보종) - 1
    for i in range(m):
        n = m - i
        지급보험금 = 0
        for j in range(n):
            지급보험금 += 일반_진전추이_보종.iloc[m-j, 3+j+1+i] - 일반_진전추이_보종.iloc[m-j, 3+j+i]
        YEAR = i + 1
        평균지급기간 = i + 0.5
        지급보험금 = max(지급보험금, 0)
        할인요소_보종.append([YEAR, 평균지급기간, 지급보험금])
    할인요소_보종 = pd.DataFrame(할인요소_보종, columns=['YEAR', '준비금_평균지급기간', '지급보험금'])
    할인요소_보종['지급보험금_비중'] = 할인요소_보종['지급보험금']/할인요소_보종['지급보험금'].sum()
    할인요소_보종['누적비율'] = 할인요소_보종['지급보험금_비중'].cumsum()
    할인요소_보종['MATURITY'] = (할인요소_보종['준비금_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소_보종.insert(0, 'PDGR_CD', PDGR_CD)
    할인요소_보종['보험료_평균지급기간'] = 할인요소_보종['준비금_평균지급기간'] - 0.2071
    할인요소_보종['조정비율'] = 0
    for i in range(m):
        if i==m-1:
            할인요소_보종['조정비율'].iloc[i] = 1
        else:
            할인요소_보종['조정비율'].iloc[i] = 할인요소_보종['누적비율'].iloc[i]+0.2071*(할인요소_보종['누적비율'].iloc[i+1]-할인요소_보종['누적비율'].iloc[i])
    할인요소_보종['1-조정비율'] = 0
    for i in range(m):
        if i==0:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]
        else:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]-할인요소_보종['조정비율'].iloc[i-1]
    할인요소_보종['MATURITY'] = np.round(할인요소_보종['보험료_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소.append(할인요소_보종)
할인요소 = pd.concat(할인요소).reset_index(drop=True)
할인요소 = 할인요소.merge(일반_상품군정보, on='PDGR_CD', how='left').drop('PDGR_CD', axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Prog

In [470]:
# 할인요소 집계
보험료부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['지급보험금_비중']*df['DISCOUNT_x']).sum())
준비금부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['1-조정비율']*df['DISCOUNT_y']).sum())
할인요소_집계 = pd.concat([보험료부채_할인요소, 준비금부채_할인요소], axis=1)
할인요소_집계.columns = ['할인요소_보험료', '할인요소_준비금']
할인요소_집계 = 할인요소_집계.reset_index()
할인요소_집계

,PDGR_NM,할인요소_보험료,할인요소_준비금
0,일반/근재,0.9708,0.9763
1,일반/기술,0.9728,0.9782
2,일반/기타,0.9847,0.9887
3,일반/배상,0.9677,0.9733
4,일반/상해,0.9863,0.9904
5,일반/종합,0.9770,0.9820
6,일반/해상,0.9815,0.9861
7,일반/화재,0.9891,0.9927


#### 7-2. 재보험자산 손실조정

In [471]:
pd.options.display.float_format = '{:,.4f}'.format

In [472]:
선도부도율_가공 = 선도부도율 >> left_join(신용등급분류, on='CRD_GRD_CD') \
    >> mutate(YEAR=X.MAT_CD.str.slice(1).astype(int)//12) \
    >> drop(['MAT_CD', 'CRD_GRD_CD', 'BASE_YYMM']) \
    >> mask(X.YEAR <= 10)

In [473]:
# 준비금부채 손실조정
손실조정_준비금부채 = 할인요소 >> select(['PDGR_NM', 'YEAR', '준비금_평균지급기간', '지급보험금_비중', 'DISCOUNT_x']) \
    >> arrange(X.PDGR_NM, -X.준비금_평균지급기간) \
    >> mutate(손실예상액=X.지급보험금_비중*0.5)
손실조정_준비금부채.columns = ['PDGR_NM', 'YEAR', '준비금_평균지급기간', '지급보험금_비중', '할인율', '손실예상액']
부도시손실액 = 손실조정_준비금부채.groupby('PDGR_NM').apply(lambda df: pd.concat([df[['PDGR_NM', '준비금_평균지급기간']], df[['손실예상액']].cumsum()], axis=1))
부도시손실액.columns = ['PDGR_NM', '준비금_평균지급기간', '부도시손실액']
손실조정_준비금부채 = 손실조정_준비금부채 >> left_join(부도시손실액, on='PDGR_NM') \
    >> arrange(X.PDGR_NM, X.준비금_평균지급기간) \
    >> left_join(선도부도율_가공, on='YEAR') \
    >> mutate(준비금_손실조정=X.FWD_PD*X.부도시손실액*X.할인율) \
    >> group_by(X.PDGR_NM, X.GRADE) \
    >> summarise(준비금_손실조정=np.sum(X.준비금_손실조정))

In [474]:
# 준비금부채 손실조정
손실조정_보험료부채 = 할인요소 >> select(['PDGR_NM', 'YEAR', '보험료_평균지급기간', '1-조정비율', 'DISCOUNT_y']) \
    >> arrange(X.PDGR_NM, -X.보험료_평균지급기간)
손실조정_보험료부채['손실예상액'] = 손실조정_보험료부채['1-조정비율']*0.5
손실조정_보험료부채.columns = ['PDGR_NM', 'YEAR', '보험료_평균지급기간', '1-조정비율', '할인율', '손실예상액']
부도시손실액 = 손실조정_보험료부채.groupby('PDGR_NM').apply(lambda df: pd.concat([df[['PDGR_NM', '보험료_평균지급기간']], df[['손실예상액']].cumsum()], axis=1))
부도시손실액.columns = ['PDGR_NM', '보험료_평균지급기간', '부도시손실액']
손실조정_보험료부채 = 손실조정_보험료부채 >> left_join(부도시손실액, on='PDGR_NM') \
    >> arrange(X.PDGR_NM, X.보험료_평균지급기간) \
    >> left_join(선도부도율_가공, on='YEAR') \
    >> mutate(보험료_손실조정=X.FWD_PD*X.부도시손실액*X.할인율) \
    >> group_by(X.PDGR_NM, X.GRADE) \
    >> summarise(보험료_손실조정=np.sum(X.보험료_손실조정))

In [475]:
# 집계
손실조정_집계 = 손실조정_준비금부채.merge(손실조정_보험료부채, on=['PDGR_NM', 'GRADE'], how='outer')
손실조정_집계

,GRADE,PDGR_NM,준비금_손실조정,보험료_손실조정
0,A,일반/근재,0.0045,0.0042
1,A+,일반/근재,0.0030,0.0029
2,A-,일반/근재,0.0067,0.0064
3,AA,일반/근재,0.0011,0.0011
4,AA+,일반/근재,0.0007,0.0006
5,AA-,일반/근재,0.0017,0.0016
6,AAA,일반/근재,0.0003,0.0003
7,B,일반/근재,0.1618,0.1558
8,BB,일반/근재,0.0484,0.0457
9,BB+,일반/근재,0.0374,0.0352


#### 8. 일반/출재/계약 - 직전 1년 경과보험료

In [476]:
# 일반 출재 계약 KICS_CNTR_CATG_NM 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 출재 계약 PDGR_NM 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 계약 PROP_YN_DVCD 가공
## 재보험특약코드 테이블을 가져오는 것과 차이가 있을 수 있음. 수정 요망.
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> mutate(RN_KND_CD=if_else(X.RRNR_TTY_CD.isin(['U2016KRQ1043001000', 'U2017KRQ1043001000', 'U2017KRS1040001000', 'U2018KRQ1043001000']), 'TP', X.RN_KND_CD)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RN_KND_CD.isin(['TN', 'FN']), '출재/비비례', '출재/비례'))
    
# 일반 출재 계약 CMSN_DVCD 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> mutate(CMSN_DVCD=if_else(X.RRNR_TTY_CD.isin(일반_연동수수료특약코드['RRNR_TTY_CD']), '연동', '비연동'))

In [477]:
# 일반 출재 계약 직전1년경과보험료 집계
일반_출재_직전1년경과보험료_집계 = 일반_출재_직전1년경과보험료_가공 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
일반_출재_직전1년경과보험료_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,RN_ELP_PRM
0,국내,비연동,출재/비례,일반/근재,"382,873.9320"
1,국내,연동,출재/비례,일반/근재,"3,331,848.5180"
2,국내,비연동,출재/비비례,일반/근재,"13,844.9250"
3,국내,비연동,출재/비례,일반/기술,"14,036,993.3310"
4,국내,연동,출재/비례,일반/기술,"2,035,802.9740"
5,국내,비연동,출재/비비례,일반/기술,"1,058,500.1870"
6,국내,비연동,출재/비례,일반/기타,"61,358,948.2910"
7,국내,비연동,출재/비례,일반/농작물재해보상,"11,846,187.9370"
8,국내,비연동,출재/비비례,일반/농작물재해보상,"811,278.4250"
9,국내,비연동,출재/비례,일반/배상,"52,656,718.8440"


#### 9. 일반/원수/계약 - 직전 1년 경과보험료

In [478]:
# 일반 출재 계약 연동수수료비중
일반_출재_직전1년경과보험료_연동수수료비중 = 일반_출재_직전1년경과보험료_집계 >> group_by(X.KICS_CNTR_CATG_NM, X.CMSN_DVCD, X.PDGR_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM))
일반_출재_직전1년경과보험료_연동수수료비중 = 일반_출재_직전1년경과보험료_연동수수료비중 >> group_by(X.PDGR_NM, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_ELP_PRM_PDGR=np.sum(X.RN_ELP_PRM)) \
    >> right_join(일반_출재_직전1년경과보험료_연동수수료비중, on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(WEIGHT=if_else(X.RN_ELP_PRM==0, 0, X.RN_ELP_PRM/X.RN_ELP_PRM_PDGR))

In [479]:
# 일반 원수 계약 KICS_CNTR_CATG_NM 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 원수 계약 PDGR_NM 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# PROP_YN_DVCD 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료_가공 >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD.isin(['02', '03']), '수재/비례', np.nan)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD=='01', '원수', X.PROP_YN_DVCD)) \
    >> mutate(PROP_YN_DVCD=if_else(X.INRI_RN_KND_CD.isin(['TN', 'FN']), '수재/비비례', X.PROP_YN_DVCD))

# 일반 원수 직전1년경과보험료 배분 (1)
일반_원수_직전1년경과보험료_배분1 = 일반_원수_직전1년경과보험료_가공 >> mask((X.RN_DVCD=='X') | (X.PROP_YN_DVCD!='원수')) \
    >> mutate(CMSN_DVCD='비연동')

# 일반 원수 직전1년경과보험료 배분 (2)
일반_원수_직전1년경과보험료_배분2 = 일반_원수_직전1년경과보험료_가공 >> mask(X.RN_DVCD=='O', X.PROP_YN_DVCD=='원수') \
    >> left_join(일반_출재_직전1년경과보험료_연동수수료비중[['KICS_CNTR_CATG_NM', 'PDGR_NM', 'CMSN_DVCD', 'WEIGHT']], on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(CMSN_DVCD=if_else(X.CMSN_DVCD.isna(), '비연동', X.CMSN_DVCD)) \
    >> mutate(WEIGHT=if_else(X.WEIGHT.isna(), 1, X.WEIGHT)) \
    >> mutate(ELP_PRM=X.ELP_PRM*X.WEIGHT)

In [480]:
# 일반 원수 직전1년경과보험료 배분
일반_원수_직전1년경과보험료_집계1 = 일반_원수_직전1년경과보험료_배분1 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM).astype(float)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

일반_원수_직전1년경과보험료_집계2 = 일반_원수_직전1년경과보험료_배분2 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

일반_원수_직전1년경과보험료_집계 = pd.concat([일반_원수_직전1년경과보험료_집계1, 일반_원수_직전1년경과보험료_집계2]) \
    >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM))
일반_원수_직전1년경과보험료_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,ELP_PRM
0,국내,비연동,수재/비례,일반/근재,"709,806.1620"
1,신흥국,비연동,수재/비례,일반/근재,"519,345.4440"
2,국내,비연동,원수,일반/근재,"1,815,582.6301"
3,미국&캐나다,비연동,원수,일반/근재,"6,952,884.1450"
4,국내,연동,원수,일반/근재,"15,248,194.4799"
5,국내,비연동,수재/비례,일반/기술,"283,297.4930"
6,기타선진국,비연동,수재/비례,일반/기술,12.7350
7,신흥국,비연동,수재/비례,일반/기술,"247,693.4950"
8,유럽&스위스,비연동,수재/비례,일반/기술,"14,171.0100"
9,중국,비연동,수재/비례,일반/기술,"12,041.0060"


#### 10. 일반/출재/보상 - 개별추산액 (2)

In [481]:
# 일반 출재 보상 KICS_CNTR_CATG_NM 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 출재 보상 PDGR_NM 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 보상 PROP_YN_DVCD 가공
## 재보험특약코드 테이블을 가져오는 것과 차이가 있을 수 있음. 수정 요망.
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> mutate(PROP_YN_DVCD='출재')
    
# 일반 출재 보상 CMSN_DVCD 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> mutate(CMSN_DVCD=if_else(X.RRNR_TTY_CD.isin(일반_연동수수료특약코드['RRNR_TTY_CD']), '연동', '비연동'))

In [482]:
# 일반 출재 보상 개별추산액 집계
일반_출재_개별추산액2_집계 = 일반_출재_개별추산액2_가공 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT).astype(float)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3)
일반_출재_개별추산액2_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,RN_OST_AMT
0,국내,비연동,출재,일반/근재,"600,636.8320"
1,국내,연동,출재,일반/근재,"1,171,089.1010"
2,국내,비연동,출재,일반/기술,"17,731,345.4260"
3,국내,연동,출재,일반/기술,"824,983.8140"
4,국내,비연동,출재,일반/기타,"6,564,877.8400"
5,국내,비연동,출재,일반/농작물재해보상,"2,394,430.0080"
6,신흥국,비연동,출재,일반/농작물재해보상,0.0000
7,국내,비연동,출재,일반/배상,"22,242,933.2240"
8,국내,연동,출재,일반/배상,"4,099,365.1240"
9,국내,비연동,출재,일반/보증,"239,149.0190"


#### 11. 일반/원수/보상 - 개별추산액 (2)

In [483]:
# 일반 출재 보상 연동수수료비중
일반_출재_개별추산액2_연동수수료비중 = 일반_출재_개별추산액2_집계 >> group_by(X.KICS_CNTR_CATG_NM, X.CMSN_DVCD, X.PDGR_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT))
일반_출재_개별추산액2_연동수수료비중 = 일반_출재_개별추산액2_연동수수료비중 >> group_by(X.PDGR_NM, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_OST_AMT_PDGR=np.sum(X.RN_OST_AMT)) \
    >> right_join(일반_출재_개별추산액2_연동수수료비중, on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(WEIGHT=if_else(X.RN_OST_AMT_PDGR==0, 0, X.RN_OST_AMT/X.RN_OST_AMT_PDGR))

In [484]:
# 일반 원수 보상 KICS_CNTR_CATG_NM 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 원수 계약 PDGR_NM 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액2_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# PROP_YN_DVCD 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액2_가공 >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD.isin(['02', '03']), '수재', np.nan)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD=='01', '원수', X.PROP_YN_DVCD))

# 일반 원수 개별추산액 배분 (1)
일반_원수_개별추산액2_배분1 = 일반_원수_개별추산액2_가공 >> mask((X.RN_DVCD=='X') | (X.PROP_YN_DVCD!='원수')) \
    >> mutate(CMSN_DVCD='비연동')

# 일반 원수 개별추산액 배분 (2)
일반_원수_개별추산액2_배분2 = 일반_원수_개별추산액2_가공 >> mask(X.RN_DVCD=='O', X.PROP_YN_DVCD=='원수') \
    >> left_join(일반_출재_직전1년경과보험료_연동수수료비중[['KICS_CNTR_CATG_NM', 'PDGR_NM', 'CMSN_DVCD', 'WEIGHT']], on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(CMSN_DVCD=if_else(X.CMSN_DVCD.isna(), '비연동', X.CMSN_DVCD)) \
    >> mutate(WEIGHT=if_else(X.WEIGHT.isna(), 1, X.WEIGHT)) \
    >> mutate(OST_AMT=X.OST_AMT*X.WEIGHT)

In [485]:
# 일반 원수 개별추산액 집계
일반_원수_개별추산액2_집계1 = 일반_원수_개별추산액2_배분1 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT).astype(float)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

일반_원수_개별추산액2_집계2 = 일반_원수_개별추산액2_배분2 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

일반_원수_개별추산액2_집계 = pd.concat([일반_원수_개별추산액2_집계1, 일반_원수_개별추산액2_집계2]) \
    >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT))
일반_원수_개별추산액2_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,OST_AMT
0,국내,비연동,수재,일반/근재,"2,086,159.3110"
1,국내,비연동,원수,일반/근재,"687,733.9352"
2,미국&캐나다,비연동,원수,일반/근재,"4,176,000.6050"
3,국내,연동,원수,일반/근재,"5,742,348.4128"
4,국내,비연동,수재,일반/기술,0.0000
5,기타선진국,비연동,수재,일반/기술,81.8280
6,신흥국,비연동,수재,일반/기술,"326,130.3940"
7,중국,비연동,수재,일반/기술,"4,140.7040"
8,국내,비연동,원수,일반/기술,"25,209,234.6012"
9,미국&캐나다,비연동,원수,일반/기술,"595,052.8200"


In [486]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.1.3'
with pd.ExcelWriter('./result/일반_현행추정부채_{}_{}.xlsx'.format(ver, now)) as writer:
    일반_원수_미경과보험료_집계.to_excel(writer, '일반_원수_미경과보험료', index=False)
    일반_출재_미경과보험료_집계.to_excel(writer, '일반_출재_미경과보험료', index=False)
    일반_납입예정보험료_집계.to_excel(writer, '일반_납입예정보험료', index=False)
    일반_원수_개별추산액_집계.to_excel(writer, '일반_원수_개별추산액', index=False)
    일반_출재_개별추산액_집계.to_excel(writer, '일반_출재_개별추산액', index=False)
    할인요소_집계.to_excel(writer, '할인요소', index=False)
    손실조정_집계.to_excel(writer, '손실조정', index=False)
    일반_출재_직전1년경과보험료_집계.to_excel(writer, '일반_출재_직전1년경과보험료', index=False)
    일반_원수_직전1년경과보험료_집계.to_excel(writer, '일반_원수_직전1년경과보험료', index=False)
    일반_출재_개별추산액2_집계.to_excel(writer, '일반_출재_개별추산액2', index=False)
    일반_원수_개별추산액2_집계.to_excel(writer, '일반_원수_개별추산액2', index=False)
    일반_원수_손해조사비율_집계.to_excel(writer, '일반_원수_손해조사비율', index=False)
    일반_출재_손해조사비율_집계.to_excel(writer, '일반_출재_손해조사비율', index=False)